## Tagebuch

### 2023-06-25

Motivierende Idee:
- Einen Graph aus Personen/Orten bauen, die in Tagesschau-Artikeln gemeinsam erwähnt wurden
- Eventuell nach Entfernung der Erwähnung gewichten
- Lustige Netzwerkanalysen im Graph durchführen
- Visualisieren!

#### Erster Ansatz

spaCy anwerfen und NER betreiben!


## Aufbauen des Datenkorpus

### Step 0: Einrichten der Umgebung

In [52]:
import json
import pandas as pd
from pathlib import Path
import spacy
from tqdm import tqdm

# Path to the root of the project
_bd = (Path(".") / "..").resolve()

# Data directory
_dd  = _bd / "data"




### Step 1: Sammeln von Nachrichtenartikeln

Artikel von [Tagesschau](https://www.tagesschau.de/) sammeln. Scraping mit [Scrapy](https://scrapy.org/):

```bash
cd newsscrape
scrapy crawl tagesspider -O ../data/tagesschau.jsonl
```

Einen Blick auf die Daten werfen:

In [53]:
df_texts = pd.read_json(open(f"{_dd}/tagesschau.jsonl", "r", encoding="utf8"), lines=True)
print(df_texts.shape)
df_texts.sample(5)

(15123, 6)


,tstamp,title,date,shorttext,url,fulltext
4729,1666153084,Wie Heimkinder sediert werden,19.10.2022 • 06:18 Uhr,"""Du wurdest mit Tabletten ruhiggestellt"" - so ...",/investigativ/br-recherche/sedierung-heimkinde...,"""Du wurdest mit Tabletten ruhiggestellt"" - so..."
7825,1671537213,Gericht in Italien erlaubt Auslieferung,20.12.2022 • 12:53 Uhr,Die Ehefrau eines ehemaligen Europaabgeordnete...,/ausland/europa/eu-korruptionsskandal-gericht-...,Die Ehefrau eines ehemaligen Europaabgeordnet...
94,1657703376,Wer entscheidet über,13.07.2022 • 11:09 Uhr,"Wenn es um Waffenexporte geht, ist eigentlich ...",/inland/innenpolitik/bundessicherheitsrat-101....,"Wenn es um Waffenexporte geht, ist eigentlich..."
592,1656911615,"""Kann keine",04.07.2022 • 07:13 Uhr,Im September läuft die Rechtsgrundlage für die...,/inland/coronavirus-lauterbach-103.html,Im September läuft die Rechtsgrundlage für di...
2251,1662043555,Herzog nimmt an Gedenkfeier teil,01.09.2022 • 16:45 Uhr,Israels Präsident Herzog nimmt bei seinem Staa...,/ausland/asien/gedenkfeier-muenchen-praesident...,Israels Präsident Herzog nimmt bei seinem Sta...


## Step 2: Extrahieren von Named Entities

Spacy vorbereiten. Wir verwenden das größte deutsche Modell.

Herausforderung I: Der erste Anlauf mit Spacys Standard-NER ergibt leider kein gutes Resultat. Spacy erkennt zwar Entitäten und ordnet sie auch häufig richtig zu, löst aber keine Ambiguitäten auf. "Obama", "Barack Obama", "Barack Hussein Obama" und "Präsident Obama" werden als vier verschiedene Entitäten erkannt. Das ist für unsere Zwecke nicht hilfreich.

Lösungsansatz: Wir verwenden ein NER-Modell, das *Entity Disambiguation* bzw. *Entity Linking* unterstützt: @sw-entity-fishing, das per @sw-spacyfishing in Spacy integriert werden kann.

In [55]:
nlp = spacy.load("de_core_news_lg")
nlp.add_pipe("entityfishing", config={"language": "de"})

In [11]:
def extract_persons(ents):
    persons = {}
    for ent in ents:
        if ent.label_:
            if ent._.kb_qid:
                persons[ent._.kb_qid] = (
                    ent.text,
                    ent.label_,
                    ent._.kb_qid,
                    ent._.url_wikidata,
                    ent._.nerd_score,
                )
    return persons


In [13]:

doc = nlp(df_texts.sample(1)["fulltext"].values[0])
print(json.dumps(extract_persons(doc.ents), indent=4))
# print(json.dumps(extract_persons(doc.ents)))
doc

{
    "Q46970": [
        "Airlines",
        "ORG",
        "Q46970",
        "https://www.wikidata.org/wiki/Q46970"
    ],
    "Q2514275": [
        "Verbraucherzentrale NRW",
        "ORG",
        "Q2514275",
        "https://www.wikidata.org/wiki/Q2514275"
    ],
    "Q819697": [
        "Handelsblatt",
        "MISC",
        "Q819697",
        "https://www.wikidata.org/wiki/Q819697"
    ],
    "Q49768": [
        "SPD",
        "ORG",
        "Q49768",
        "https://www.wikidata.org/wiki/Q49768"
    ],
    "Q49762": [
        "CDU",
        "ORG",
        "Q49762",
        "https://www.wikidata.org/wiki/Q49762"
    ],
    "Q1197": [
        "nieders\u00e4chsischen",
        "MISC",
        "Q1197",
        "https://www.wikidata.org/wiki/Q1197"
    ],
    "Q1502854": [
        "Steffi Lemke",
        "PER",
        "Q1502854",
        "https://www.wikidata.org/wiki/Q1502854"
    ],
    "Q49766": [
        "Gr\u00fcne",
        "ORG",
        "Q49766",
        "https://www.wiki

 Flugreisende müssen schon bei der Buchung den vollen Ticketpreis zahlen. Verbraucherschützer machen sich angesichts chaotischer Zustände in der Branche dafür stark, das zu ändern. Die Airlines halten dagegen.  Verbraucherschützer fordern ein Ende der Praxis, dass Flugpassagiere schon bei Buchung ihrer Reise den vollen Ticketpreis bezahlen müssen. "Passagiere sind es leid, den Airlines zinslose Kredite zu geben, bei abgesagten Flügen auf den Kosten sitzen zu bleiben oder im schlimmsten Fall das Risiko einer Insolvenz tragen zu müssen", sagte Marion Jungbluth vom Bundesverband der Verbraucherzentralen (VZBV) dem "Handelsblatt". Von der Bundesregierung verlange man, die Vorkassepraxis zu reformieren. Gezahlt werden solle künftig erst beim Check-in. Auch das von SPD und CDU regierte Land Niedersachsen hat sich mit einer Bundesratsinitiative den Vorstoß der VZBV zu eigen gemacht. Unterstützung hat zudem das grün geführte Bundesministerium für Umwelt und Verbraucherschutz signalisiert. Die 

Problem: Ziemlich viele Erdogans...wir brauchen ein Verfahren, um
unterschiedlichen Schreibweisen dieselbe Entität zuzuordnen.

- **Entity Linking**
  - spaCy hat sowas, aber man braucht: 
    - eine Knowledge Base
    - eine Funktion, die plausible Kandidaten aus der KB zieht,
    - ein ML-Modell, das anhand des lokalen Kontexts den wahrscheinlichsten
      Kandidaten auswählt
    - Bisschen viel
  - Recherche:
    - [amazon-science/ReFinED: ReFinED is an efficient and accurate entity linking (EL) system.](https://github.com/amazon-science/ReFinED)
    - [Improving Named Entity Disambiguation using Entity Relatedness within Wikipedia | by Will Seaton | Towards Data Science](https://towardsdatascience.com/improving-named-entity-disambiguation-using-entity-relatedness-within-wikipedia-92f400ee5994)
    - [Introducing the Kensho Derived Wikimedia Dataset | by Gabriel Altay | Kensho Blog](https://blog.kensho.com/announcing-the-kensho-derived-wikimedia-dataset-5d1197d72bcf)
    - [Kensho Derived Wikimedia Dataset | Kaggle](https://www.kaggle.com/datasets/kenshoresearch/kensho-derived-wikimedia-data?select=property_aliases.csv)
    - [kdwd_wikidata_introduction | Kaggle](https://www.kaggle.com/code/kenshoresearch/kdwd-wikidata-introduction)
- **Naiver Ansatz**
  - Nur das letzte Wort der Entität

## Scraping

**Quelle**: https://www.tagesschau.de/archiv?datum=YYYY-MM-DD -> z.B.: https://www.tagesschau.de/archiv?datum=2023-06-22

div.teasergroup:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > a:nth-child(1)
html body div.global-wrapper main.content-wrapper.content-wrapper--show-cuts div.layout-container div#content.layout-content.scrolled div.container div.columns.twelve.teasergroup div.columns.twelve div.columns.twelve.m-ten.m-offset-one.l-eight.l-offset-two div.copytext-element-wrapper__vertical-only div.teaser-right.twelve a.teaser-right__link

In [3]:
def extract_entities(text):
    doc = nlp(text)
    entities = {}
    for ent in doc.ents:
        # if ent.label_ in ["PER"]:
        if ent.label_:
            if ent._.kb_qid:
                entities[ent._.kb_qid] = (
                    ent.text,
                    ent.label_,
                    ent._.kb_qid,
                    ent._.url_wikidata,
                    ent._.nerd_score,
                )
    return entities


In [21]:
print(df_texts["title"].unique().shape, df_texts.shape)

df_texts.sort_values("url", inplace=True)
df_texts

(14883,) (15123, 6)


,tstamp,title,date,shorttext,url,fulltext
12771,1682418239,Neuer Livestream und verbesserter Audio-Player,25.04.2023 • 12:23 Uhr,Mit dem jüngsten Update der tagesschau-App bie...,/app,Mit dem jüngsten Update der tagesschau-App bi...
11007,1680256839,Quote gegen,31.03.2023 • 12:00 Uhr,Seit einem Jahr gilt in Argentiniens Behörden ...,/argentinien-transmenschen-101.html,Seit einem Jahr gilt in Argentiniens Behörden...
258,1657270166,Schock und Trauer über Attentat,08.07.2022 • 10:49 Uhr,Das Attentat auf Japans Ex-Ministerpräsidenten...,/ausland/abe-anschlag-reaktion-101.html,Das Attentat auf Japans Ex-Ministerpräsidente...
696,1656799826,Palästinenser übergeben tödliche Kugel an USA,03.07.2022 • 00:10 Uhr,Im Fall der im Westjordanland getöteten Journa...,/ausland/abuakle-kugel-palaestinenser-101.html,Im Fall der im Westjordanland getöteten Journ...
1424,1660519926,Kopten nach Brand in Kirche unter Schock,15.08.2022 • 01:32 Uhr,Ein Feuer in einer koptischen Kirche hat zahlr...,/ausland/aegypten-kopten-feuer-101.html,Ein Feuer in einer koptischen Kirche hat zahl...
...,...,...,...,...,...,...
9756,1677002967,Das Universum im Labor,21.02.2023 • 19:09 Uhr,"In Darmstadt wird ein neuer, großer Teilchenbe...",/wissen/teilchenbeschleuniger-darmstadt-101.html,"In Darmstadt wird ein neuer, großer Teilchenb..."
11895,1681408814,Wie Bären der,13.04.2023 • 20:00 Uhr,"Wenn Menschen lange bettlägerig sind, wächst d...",/wissen/thromboseforschung-baeren-101.html,"Wenn Menschen lange bettlägerig sind, wächst ..."
8435,1673547931,2022 war eines der weltweit wärmsten Jahre,12.01.2023 • 19:25 Uhr,Das vergangene Jahr war weltweit das fünft- od...,/wissen/weltwetterorganisation-waerme-2022-101...,Das vergangene Jahr war weltweit das fünft- o...
13100,1684569237,Wie geht es den Wildbienen?,20.05.2023 • 09:53 Uhr,Während die Honigbiene so gut dasteht wie lang...,/wissen/wildbienen-100.html,Während die Honigbiene so gut dasteht wie lan...


In [ ]:

with open("entities.jsonl", "w", encoding="utf8") as f:
    for text in tqdm(df_texts["fulltext"].values):
        entities = extract_entities(text)
        if entities:
            json.dump({"entities": entities}, f, ensure_ascii=False)
            f.write("\n")

Der Shit ist lahm, ich brauche mehr Parallelität!

In [42]:
# cluster = LocalCluster()
# client = Client(cluster)
# client

c:\Users\p\miniconda3\envs\mbb-asud-proto-tagesschau\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56588 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:56588/status,
Dashboard: http://127.0.0.1:56588/status,Workers: 4
Total threads: 12,Total memory: 15.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56590,Workers: 4
Dashboard: http://127.0.0.1:56588/status,Total threads: 12
Started: Just now,Total memory: 15.90 GiB
Comm: tcp://127.0.0.1:56625,Total threads: 3
Dashboard: http://127.0.0.1:56630/status,Memory: 3.98 GiB
Nanny: tcp://127.0.0.1:56593,


In [51]:
df_text_nrm = pd.DataFrame(df_texts.sample(10).loc[:, "fulltext"].str.normalize("NFKD"))
df_text_nrm

,fulltext
9303,Auch nach Tagen sorgt die Aussage von Außenmi...
2052,"Die USA haben eigenen Angaben nach eine ""Anti..."
9938,In Jerusalem haben Tausende Menschen vor dem ...
185,Seit Ausbruch des Ukraine-Kriegs ist Gas knap...
10477,Im Kabinett hat eine Nationale Wasserstrategi...
9824,"Er hat gemahnt, gewarnt, gefleht. Seine Auftr..."
6670,Die Themenliste des EU-Gipfels war lang und d...
14885,Die ukrainische Gegenoffensive hat begonnen u...
11004,"Russlands Präsident Putin, der vor Chinas St..."
6652,Besorgnis und Kritik nach der Sperrung mehrer...


In [11]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

def do_extract_entities(df, partition_info: dict):
    with open(f"../out/entities-{partition_info['number']}.jsonl", "w", encoding="utf8") as f:    
        for text in df["fulltext"].values:
            entities = extract_entities(text)
            if entities:
                json.dump({"entities": entities}, f, ensure_ascii=False)
                f.write("\n")

ddf_text = dd.from_pandas(df_texts, chunksize=200)

with ProgressBar():
    ddf_text.map_partitions(do_extract_entities, meta=("entities", "object")).compute()

[########################################] | 100% Completed | 2hr 7ms


In [22]:
from SPARQLWrapper import SPARQLWrapper, JSON
import sys

user_agent = f"Academic Research Python/{sys.version_info[0]}.{sys.version_info[0]}"
endpoint = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(endpoint, agent=user_agent, returnFormat=JSON)

sparql.setQuery("""
""")

### Entities von Wikidata scrapen

1. Alle `entities-*.jsonl` zusammenfügen
2. Entities in neues Dict, dabei Infos aus der NER beibehalten und Anreicherung durch Wikidata vorbereiten
3. Liste aller Entities für Scraper generieren (in jsonl oder so)
4. Alle Entities von Wikidata scrapen
   1. Herausforderung I: Manche Einträge haben kein Label/keine Description -> dann statt dessen Entitätskenner
   2. Herausforderung II: Manche Einträge leiten auf andere Entitäten weiter -> dann weitergeleitete ID speichern
5. Zusammenführen (direkt im Scraper?)

In [26]:
from pathlib import Path

with open("../out/all-entities.jsonl", mode="w", encoding="utf8") as f_out:
    for entfile in Path("../out/").glob("entities-*.jsonl"):
        with open(entfile, mode="r", encoding="utf8") as f_in:
            f_out.write(f_in.read())

In [45]:
import jsonlines as jl

catalog = {}
with jl.open("../out/all-entities.jsonl", mode="r") as all_objs:
    for obj in tqdm(all_objs.iter()):
        for key, extract in obj["entities"].items():
            if key not in catalog:
                catalog[key] = {
                    "id": key,
                    "extracts": [extract],
                }
            else:
                catalog[key]["extracts"].append(extract)

# json.dump(catalog, open("../out/catalog.json", mode="w"), indent=4, ensure_ascii=True)

with jl.open("../out/catalog.jsonl", mode="w") as catalog_file:
    for k, v in tqdm(catalog.items()):
        catalog_file.write(v)

# catalog = jl.open("../out/catalog.jsonl", mode="r")


14957it [00:01, 10660.68it/s]
100%|██████████| 36016/36016 [00:00<00:00, 55748.89it/s]
